# Database Setup

This notebook is intended to setup a MongoDB and a MySql databases.

Run this Notebook only once to create the required tables and collections.

In [3]:
!pip install -r "./requirements_db.txt"

In [38]:
import datetime

import os
from pathlib import Path
import pandas as pd

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

import pymysql 

from sqlalchemy import create_engine 

# PART I - MongoDB Setup

Although Mongo would create the Database and Collections when I first insert data into it, I choose to create the collections beforehand.

## Connect to MongoDB instance

In [5]:
MONGOURI = os.getenv('MONGOURI')
MONGODB = os.getenv('MONGODB')

client = MongoClient(MONGOURI, server_api=ServerApi('1'))

# Select database by name.
# If database does not exist it will create a new one.
db = client[MONGODB]  

## Create the required Collections

In [8]:
stockprice = db.create_collection("stockprice")
stocktweet = db.create_collection("stocktweet")

# PART II - MySQL Setup

## 1. Connect to the MySQL instance

In [27]:
# Get the data from the environment variables
hostname = os.getenv('MYSQLHOST')
port=int(os.getenv('MYSQLPORT'))
username = os.getenv('MYSQLUSR')
password = os.getenv('MYSQLPASS')
database_name = os.getenv('MYSQLDB') 
ca_cert_path = '../ca.pem'

# Create MySQL connection object
connection = pymysql.connect( 
    host=hostname, 
    port=port, 
    user=username, 
    password=password, 
    database=database_name, 
    ssl={'ca': ca_cert_path} 
)

In [29]:
# Create the SQLAlchemy engine for MySQL
engine = create_engine(f"mysql+pymysql://{username}:{password}@{hostname}:{port}/{database_name}")

## 2. Define table names

In [32]:
tables = [
    "Stocktweet",
    "Stockprice",
    "Sentiment",
    "Company",
]

## 3. Drop tables if exists

In [33]:
# Drop all tables
for table in tables:
    query = "DROP TABLE IF EXISTS {};".format(table)

    with connection.cursor() as cursor:
        cursor.execute(query)
        print("Table '{}' dropped successfully".format(table))



Table 'Stocktweet' dropped successfully
Table 'Stockprice' dropped successfully
Table 'Sentiment' dropped successfully
Table 'Company' dropped successfully


## 4. Create the tables

In [34]:
# Query
table_queries = [
    """
    CREATE TABLE IF NOT EXISTS Company (
        ticker VARCHAR(6) PRIMARY KEY,
        name VARCHAR(80)
    );
    """,
    
    """
    CREATE TABLE IF NOT EXISTS Stocktweet (
        id INT PRIMARY KEY,
        date DATE,
        ticker VARCHAR(6),
        tweet TEXT,
        CONSTRAINT fk_ticker_stocktweet
            FOREIGN KEY (ticker) REFERENCES Company(ticker)
            ON DELETE CASCADE
    );
    """,
          
    """
    CREATE TABLE IF NOT EXISTS Stockprice (
        id INT PRIMARY KEY AUTO_INCREMENT,
        ticker VARCHAR(6),
        Date DATE,
        Open DECIMAL(12, 6),
        High DECIMAL(12, 6),
        Low DECIMAL(12, 6),
        Close DECIMAL(12, 6),
        AdjClose DECIMAL(12, 6),
        Volume BIGINT,
        CONSTRAINT fk_ticker_stockprice
            FOREIGN KEY (ticker) REFERENCES Company(ticker)
            ON DELETE CASCADE
    );
    """,
    
    """
    CREATE TABLE IF NOT EXISTS Sentiment (
        id INT PRIMARY KEY AUTO_INCREMENT,
        ticker VARCHAR(6),
        date DATE,
        score DECIMAL(12,6),
        CONSTRAINT fk_ticker_sentiment
            FOREIGN KEY (ticker) REFERENCES Company(ticker)
            ON DELETE CASCADE
    );
    """
]

for table_query in table_queries:
    with connection.cursor() as cursor:
        cursor.execute(table_query)
        print("Table created successfully.")

Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.


# PART III - Load Data

# A. Load Data into MySQL

## 1. Company Tickers

In [35]:
tickers = [
    "AAPL",
    "AMT",
    "AMZN",
    "BA",
    "BABA",
    "BAC",
    "BKNG",
    "BRK.A",
    "BRK.B",
    "CCL",
    "CVX",
    "DIS",
    "FB",
    "GOOG",
    "GOOGL",
    "HD",
    "JNJ",
    "JPM",
    "KO",
    "LOW",
    "MA",
    "MCD",
    "META",
    "MSFT",
    "NFLX",
    "NKE",
    "NVDA",
    "PFE",
    "PG",
    "PYPL",
    "SBUX",
    "TM",
    "TSLA",
    "TSM",
    "UNH",
    "UPS",
    "V",
    "WMT",
    "XOM"
]

# Insert multiple rows of data
insert_query = '''
INSERT INTO Company (ticker)
VALUES (%s);
'''


with connection.cursor() as cursor:
    cursor.executemany(insert_query, tickers)
    connection.commit()
    print("Tickers inserted successfully!")


Tickers inserted successfully!


# 2. Stocktweet

In [36]:
# Define the data types for stocktweet columns
dtype_dict = {
    'id': 'int64',           
    'ticker': 'str',        
    'tweet': 'str'
}

# Load stocktweet
df = pd.read_csv('./data/stocktweet/stocktweet.csv', dtype=dtype_dict, parse_dates=['date'])

# format date column
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Insert the data into the MySQL table
df.to_sql("Stocktweet", engine, if_exists='append', index=False)

print("Stocktweet loaded into MySQL table successfully.")

Stocktweet loaded into MySQL table successfully.


# 3. Stockprice

In [37]:
# Define the data types for stockprice columns
dtype_dict = {       
    'Open': 'float',    
    'High': 'float',
    'Low': 'float',
    'Close': 'float',
    'Adj Close': 'float',
    'Volume': 'int'
}

In [40]:


folder_path = Path('./data/stockprice/')

for file_path in folder_path.iterdir():
    if (file_path.is_file()) and (file_path.name.rstrip(".csv").replace("-",".") in tickers):  # Check if it’s a file and a ticker exists
        print(file_path.name.rstrip(".csv"))
        df = pd.read_csv("./data/stockprice/{}".format(file_path.name), dtype=dtype_dict, parse_dates=['Date'])
        
        # Rename column 'Adj Close' to 'AdjClose'
        df.rename(columns={'Adj Close': 'AdjClose'}, inplace=True)
        
        df['ticker'] = file_path.name.rstrip(".csv").replace("-", ".")
        
        # format date column
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

        df.to_sql("Stockprice", engine, if_exists='append', index=False)

ABNB
MA
UPS
BRK-A
BAC
KO
TM
MSFT
BA
BKNG
FB
PG
AMZN
BRK-B
XOM
NFLX
LOW
SBUX
META
MCD
CCL
V
UNH
TSLA
AAPL
NKE
BABA
GOOGL
PFE
HD
CVX
NVDA
WMT
PYPL
DIS
JNJ
AMT
GOOG
JPM
TSM
